In [37]:
%pip install tianshou
%pip install pygame

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=28151) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gymnasium as gym
from torch.utils.tensorboard import SummaryWriter

import tianshou as ts
from tianshou.policy import DQNPolicy

In [39]:
class CBPLayer(nn.Module):
    def __init__(self, in_features, out_features, activation=nn.ReLU(), cbp_params=None):
        """
        in_features: number of inputs.
        out_features: number of neurons (features) in this layer.
        activation: activation function.
        cbp_params: dict with keys:
            - eta: decay rate (e.g. 0.99)
            - rho: replacement rate (e.g. 1e-4)
            - m: maturity threshold (e.g. 100)
        """
        super().__init__()
        # Initialized the weight for each layer (Initialize: Initialize the weights w0, ..., wL. Let, wl be sampled from a distribution dl)
        self.linear = nn.Linear(in_features, out_features)
        self.activation = activation
        
        # Set default CBP parameters if none provided.
        if cbp_params is None:
            cbp_params = {'eta': 0.99, 'rho': 1e-4, 'm': 100}
        self.eta = cbp_params.get('eta', 0.99)
        self.rho = cbp_params.get('rho', 1e-4)
        self.m = cbp_params.get('m', 100)
        
        # Initialize: Utilities u1, ..., uL, average feature activation f1, ..., fl
        # Then, register_buffer is the PyTorch func to ensure these tensor are moved to the proper CPU or GPU we plan to use
        self.register_buffer('age', torch.zeros(out_features))
        self.register_buffer('u', torch.zeros(out_features))      # overall utility
        self.register_buffer('z', torch.zeros(out_features))      # mean-corrected contribution utility
        self.register_buffer('f_hat', torch.zeros(out_features))  # running average activation
        
        self.last_activation = None  # will be set in forward()

    # this is part of the the forward pass (put here and will be called in CBPNetwork forward pass to avoid redudnacy and readability)
    def forward(self, x):
        z = self.linear(x)
        a = self.activation(z)
        # Function 4 (in the curent paper it's function 1 but in the YT video it's func 4) in the paper discuss about 
        # tracking feature activations to compute utilities. However, it does not specify how to store activations in an implementation.
        # last_activation will be used below to track feature activations to compute utilities
        # detach() tell PyTorch to not track gradients for last_activation (since we only need the activation values, not their gradients) 
        # to avoid using unnecessary memory
        self.last_activation = a.detach()
        return a

    def update_cbp(self, next_layer_weight=None):
        """
        This function updates the CBP utilities for this layer and reinitialize a fraction of neurons if needed.
        Paper: for each input x_t do
        """
        eta = self.eta
        rho = self.rho
        m = self.m
        
        # If no forward pass has been done, skip the update.
        if self.last_activation is None:
            return None

        # Increase the age (Paper: Update age: al += 1).
        self.age += 1

        # Compute the batch-average activation per neuron.
        h = self.last_activation.mean(dim=0)
        
        # Paper: Update feature utility: Using Function 4, 5, and 6
        # Update the running average (f_hat) using an exponential moving average.
        beta = 0.01
        # Paper: Function 2
        self.f_hat = (1 - beta) * self.f_hat + beta * h

        # Compute the absolute difference |h - f_hat| part in function 4 below
        diff = (h - self.f_hat).abs()

        # Sum over the absolute weights connecting this neuron to the next layer.
        if next_layer_weight is not None:
            # next_layer_weight: shape (n_next, out_features) — sum over next-layer neurons.
            sum_abs_out = next_layer_weight.abs().sum(dim=0)
        else:
            sum_abs_out = torch.zeros_like(self.u)
        
        # Sum over the absolute input weights for each neuron in this layer.
        # self.linear.weight is shape (out_features, in_features)
        sum_abs_in = self.linear.weight.abs().sum(dim=1)  # one value per neuron

        # --- Function 4: Mean-corrected Contribution Utility ---
        self.z = (1 - eta) * diff * sum_abs_out + eta * self.z

        # --- Function 5: Adaptation Utility ---
        y = (diff * sum_abs_out) / (sum_abs_in + 1e-8)  # Here, we add epsilon (1e-8) to avoid division by zero

        # --- Function 6: Overall Feature Utility ---
        self.u = (1 - eta) * y + eta * self.u

        # Find eligible features: Features with age more than m (age > m)
        # This createes a boolean tensor (True for neurons older than m and False otherwise).
        eligible = self.age > m

        # If no neuron are eligible for replacement, then skip this computation to avoid error
        if eligible.sum() > 0:
            eligible_indices = torch.nonzero(eligible).squeeze()
            # Determine the number of neurons to replace.
            n_eligible = eligible.sum().item()
            n_replace = max(1, int(n_eligible * rho))
            # Paper: Features to replace: nl∗ρ of eligible features with smallest utility, let their indices be r
            # Among the eligible neurons, select those with the smallest utility.
            # The argsort function call will return indices that would sort the tensor in ascending order.
            utilities = self.u[eligible]
            sorted_order = utilities.argsort() # Sort by utility
            # Map sorted indices back to global indices.
            if eligible_indices.dim() == 0:
                eligible_indices = eligible_indices.unsqueeze(0)
            replace_indices = eligible_indices[sorted_order[:n_replace]] # Here, we select lowest utility
            # For each neuron to replace, we reinitialize its input weights and reset its buffers.
            for idx in replace_indices:
                fan_in = self.linear.in_features
                bound = 1 / math.sqrt(fan_in)

                # Paper: Initialize input weights: Reset the input weights wl−1[r] using samples from dl
                # Reinitialize the weights for this neuron (i.e. one row of self.linear.weight)
                with torch.no_grad():
                    self.linear.weight[idx].uniform_(-bound, bound)
                    if self.linear.bias is not None:
                        self.linear.bias[idx].uniform_(-bound, bound)
                # Paper: Initialize utility, feature activation, and age
                self.u[idx] = 0
                self.z[idx] = 0
                self.f_hat[idx] = 0
                self.age[idx] = 0
            # Return the indices of the replaced neurons so that the next layer can zero out its input weights.
            return replace_indices
        return None

In [40]:
class CBPNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_sizes, cbp_params=None):
        """
        input_dim: dimensionality of the input.
        output_dim: number of actions (output neurons).
        hidden_sizes: list of hidden layer sizes.
        cbp_params: dictionary of CBP parameters (eta, rho, m).
        """
        # Iniitialize the layers
        super().__init__()
        layers = []
        in_dim = input_dim
        for hidden_dim in hidden_sizes:
            layers.append(CBPLayer(in_dim, hidden_dim, activation=nn.ReLU(), cbp_params=cbp_params))
            in_dim = hidden_dim
        self.hidden_layers = nn.ModuleList(layers)
        self.out_layer = nn.Linear(in_dim, output_dim)

    # Forward pass
    def forward(self, obs, state=None, info=None):
        # This will ensure we're working with a tensor
        if isinstance(obs, np.ndarray):
            obs = torch.from_numpy(obs).float()
        
        # Handle batch vs single observation
        if obs.dim() == 1:
            obs = obs.unsqueeze(0)  # Add batch dimension
            
        x = obs
        for layer in self.hidden_layers:
            x = layer(x)
        logits = self.out_layer(x)
        
        # Finally, return both logits and None for the state
        # This will match what Tianshou expects for a stateless network
        return logits, state

    def update_cbp(self):
        """
        Call update_cbp() on each CBP layer. For each layer, if some neurons were replaced,
        zero out the corresponding incoming weights in the next layer.
        """
        # Iterate through each layer
        for i, layer in enumerate(self.hidden_layers):
            if i < len(self.hidden_layers) - 1:
                next_weight = self.hidden_layers[i + 1].linear.weight
            else:
                next_weight = self.out_layer.weight  # for the last hidden layer, use output layer weights.
            # Call update_cbp func of each layer to update the paper's tasks (comment in the update_cbp func above already)
            replace_indices = layer.update_cbp(next_layer_weight=next_weight)
            if replace_indices is not None:
                with torch.no_grad():
                    # For next layer, set weights corresponding to replaced features to zero.
                    # Paper: "Initialize output weights: Set wl[r] to zero"
                    next_weight[:, replace_indices] = 0

In [41]:
class CBPDQNPolicy(DQNPolicy):
    def learn(self, batch, **kwargs):
        # This learn function will call DQNPolicy.learn() from Tianshou, this will:
        # Run the Runs the forward pass using CBPNetwork.forward().
        # Then, it will computes loss.
        # Finally, it updates model weights (SGD).
        result = super().learn(batch, **kwargs)
        # Then update the CBP utilities then (if needed) replace features.
        if hasattr(self.model, 'update_cbp'):
            self.model.update_cbp()
        return result

In [42]:
import numpy as np

# --- Set up logging ---
logger = ts.utils.TensorboardLogger(SummaryWriter("logs/cbp_dqn"))

# --- Create the environment ---
env_name = "CartPole-v1"
train_env = gym.make(env_name)
test_env = gym.make(env_name)

# --- Get state and action info ---
obs_shape = train_env.observation_space.shape  
n_actions = train_env.action_space.n           
input_dim = np.prod(obs_shape)

# --- Define CBP parameters ---
cbp_params = {
    'eta': 0.99,   # decay rate 
    'rho': 1e-4,   # replacement rate 
    'm': 100       # maturity threshold 
}

# --- Build the CBP network ---
hidden_sizes = [128, 128, 128]
model = CBPNetwork(input_dim=input_dim, output_dim=n_actions, hidden_sizes=hidden_sizes, cbp_params=cbp_params)

# --- Set up optimizer ---
optim = torch.optim.Adam(model.parameters(), lr=0.001)

# --- Create the CBP-DQN policy ---
policy = CBPDQNPolicy(
    model=model,
    optim=optim,
    discount_factor=0.9,
    estimation_step=3,
    target_update_freq=320,
    action_space=train_env.action_space
)

# --- Set up data collectors ---
train_collector = ts.data.Collector(
    policy, 
    train_env, 
    ts.data.VectorReplayBuffer(20000, 1),
    exploration_noise=True
)
test_collector = ts.data.Collector(policy, test_env, exploration_noise=True)

# --- Train the agent ---
result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=5,
    step_per_epoch=10000,
    step_per_collect=10,
    episode_per_test=100,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=logger,
).run()

print(f"Finished training in {result.timing.total_time} seconds")
torch.save(policy.state_dict(), 'models/cbp_dqn.pth')

# --- Evaluate the trained agent ---
policy.load_state_dict(torch.load('models/cbp_dqn.pth'))
policy.eval()
policy.set_eps(0.05)

# Create an environment with rendering enabled.
eval_env = gym.make(env_name, render_mode="human")
eval_env.reset()
collector = ts.data.Collector(policy, eval_env, exploration_noise=True)
collector.collect(n_episode=1, render=1/35, reset_before_collect=True)
eval_env.close()

Epoch #1: 10001it [00:08, 1127.31it/s, env_step=10000, gradient_step=1000, len=10, n/ep=1, n/st=10, rew=10.00]                           


Epoch #1: test_reward: 9.880000 ± 1.107068, best_reward: 9.880000 ± 1.107068 in #1


Epoch #2: 10001it [00:10, 911.97it/s, env_step=20000, gradient_step=2000, len=22, n/ep=0, n/st=10, rew=22.00]                             


Epoch #2: test_reward: 34.170000 ± 5.770711, best_reward: 34.170000 ± 5.770711 in #2


Epoch #3: 10001it [00:09, 1015.40it/s, env_step=30000, gradient_step=3000, len=102, n/ep=0, n/st=10, rew=102.00]                           


Epoch #3: test_reward: 27.580000 ± 4.164565, best_reward: 34.170000 ± 5.770711 in #2


Epoch #4: 10001it [00:16, 600.20it/s, env_step=40000, gradient_step=4000, len=144, n/ep=0, n/st=10, rew=144.00]                            


Epoch #4: test_reward: 407.370000 ± 42.719938, best_reward: 407.370000 ± 42.719938 in #4


Epoch #5: 10001it [00:10, 924.49it/s, env_step=50000, gradient_step=5000, len=154, n/ep=0, n/st=10, rew=154.00]                           


Epoch #5: test_reward: 29.930000 ± 8.510294, best_reward: 407.370000 ± 42.719938 in #4
Finished training in 71.10224294662476 seconds


In [43]:
import numpy as np

# --- Set up logging ---
logger = ts.utils.TensorboardLogger(SummaryWriter("logs/cbp_dqn"))

# --- Create the environment ---
env_name = "CartPole-v1"
train_env = gym.make(env_name)
test_env = gym.make(env_name)

# --- Get state and action info ---
obs_shape = train_env.observation_space.shape  
n_actions = train_env.action_space.n           
input_dim = np.prod(obs_shape)

# --- Define CBP parameters ---
cbp_params = {
    'eta': 0.99,   # decay rate 
    'rho': 1e-4,   # replacement rate 
    'm': 100       # maturity threshold 
}

# --- Build the CBP network ---
hidden_sizes = [128, 128, 128]
model = CBPNetwork(input_dim=input_dim, output_dim=n_actions, hidden_sizes=hidden_sizes, cbp_params=cbp_params)

# --- Set up optimizer ---
optim = torch.optim.Adam(model.parameters(), lr=0.001)

# --- Create the CBP-DQN policy ---
policy = CBPDQNPolicy(
    model=model,
    optim=optim,
    discount_factor=0.9,
    estimation_step=3,
    target_update_freq=320,
    action_space=train_env.action_space
)

# --- Set up data collectors ---
train_collector = ts.data.Collector(
    policy, 
    train_env, 
    ts.data.VectorReplayBuffer(20000, 1),
    exploration_noise=True
)
test_collector = ts.data.Collector(policy, test_env, exploration_noise=True)

# --- Train the agent ---
result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=5,
    step_per_epoch=10000,
    step_per_collect=10,
    episode_per_test=100,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=logger,
).run()

print(f"Finished training in {result.timing.total_time} seconds")
torch.save(policy.state_dict(), 'models/cbp_dqn.pth')

# --- Evaluate the trained agent ---
policy.load_state_dict(torch.load('models/cbp_dqn.pth'))
policy.eval()
policy.set_eps(0.05)

# Create an environment with rendering enabled.
eval_env = gym.make(env_name, render_mode="human")
eval_env.reset()
collector = ts.data.Collector(policy, eval_env, exploration_noise=True)
collector.collect(n_episode=1, render=1/35, reset_before_collect=True)
eval_env.close()

Epoch #1: 10001it [00:08, 1147.30it/s, env_step=10000, gradient_step=1000, len=10, n/ep=1, n/st=10, rew=10.00]                           


Epoch #1: test_reward: 9.570000 ± 0.886059, best_reward: 9.690000 ± 1.074197 in #0


Epoch #2: 10001it [00:09, 1023.77it/s, env_step=20000, gradient_step=2000, len=10, n/ep=0, n/st=10, rew=10.50]                           


Epoch #2: test_reward: 9.920000 ± 1.110675, best_reward: 9.920000 ± 1.110675 in #2


Epoch #3: 10001it [00:09, 1055.48it/s, env_step=30000, gradient_step=3000, len=12, n/ep=0, n/st=10, rew=12.00]                           


Epoch #3: test_reward: 36.390000 ± 8.984314, best_reward: 36.390000 ± 8.984314 in #3


Epoch #4: 10001it [00:09, 1053.41it/s, env_step=40000, gradient_step=4000, len=13, n/ep=1, n/st=10, rew=13.00]                            


Epoch #4: test_reward: 14.260000 ± 1.752826, best_reward: 36.390000 ± 8.984314 in #3


Epoch #5: 10001it [00:09, 1019.15it/s, env_step=50000, gradient_step=5000, len=16, n/ep=1, n/st=10, rew=16.00]                            


Epoch #5: test_reward: 18.720000 ± 2.474187, best_reward: 36.390000 ± 8.984314 in #3
Finished training in 50.10233783721924 seconds


In [44]:
import numpy as np

# --- Set up logging ---
logger = ts.utils.TensorboardLogger(SummaryWriter("logs/cbp_dqn"))

# --- Create the environment ---
env_name = "CartPole-v1"
train_env = gym.make(env_name)
test_env = gym.make(env_name)

# --- Get state and action info ---
obs_shape = train_env.observation_space.shape  
n_actions = train_env.action_space.n           
input_dim = np.prod(obs_shape)

# --- Define CBP parameters ---
cbp_params = {
    'eta': 0.99,   # decay rate 
    'rho': 1e-4,   # replacement rate 
    'm': 100       # maturity threshold 
}

# --- Build the CBP network ---
hidden_sizes = [128, 128, 128]
model = CBPNetwork(input_dim=input_dim, output_dim=n_actions, hidden_sizes=hidden_sizes, cbp_params=cbp_params)

# --- Set up optimizer ---
optim = torch.optim.Adam(model.parameters(), lr=0.001)

# --- Create the CBP-DQN policy ---
policy = CBPDQNPolicy(
    model=model,
    optim=optim,
    discount_factor=0.975,
    estimation_step=3,
    target_update_freq=1000,
    action_space=train_env.action_space
)

# --- Set up data collectors ---
train_collector = ts.data.Collector(
    policy, 
    train_env, 
    ts.data.VectorReplayBuffer(20000, 1),
    exploration_noise=True
)
test_collector = ts.data.Collector(policy, test_env, exploration_noise=True)

# --- Train the agent ---
best_mean_reward = -np.inf 

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(policy.state_dict(), 'models/cbp_dqn.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=15,
    step_per_epoch=10000,
    step_per_collect=10,
    episode_per_test=100,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=logger,
).run()

save_best_model(result.best_reward)

print(f"Finished training in {result.timing.total_time} seconds")

# --- Evaluate the trained agent ---
policy.load_state_dict(torch.load('models/cbp_dqn.pth'))
policy.eval()
policy.set_eps(0.05)

# Create an environment with rendering enabled.
eval_env = gym.make(env_name, render_mode="human")
eval_env.reset()
collector = ts.data.Collector(policy, eval_env, exploration_noise=True)
collector.collect(n_episode=1, render=1/35, reset_before_collect=True)
eval_env.close()

Epoch #1: 10001it [00:09, 1020.82it/s, env_step=10000, gradient_step=1000, len=9, n/ep=1, n/st=10, rew=9.00]                           


Epoch #1: test_reward: 9.610000 ± 0.893252, best_reward: 9.610000 ± 0.893252 in #1


Epoch #2: 10001it [00:09, 1003.13it/s, env_step=20000, gradient_step=2000, len=10, n/ep=1, n/st=10, rew=10.00]                           


Epoch #2: test_reward: 9.590000 ± 0.917551, best_reward: 9.610000 ± 0.893252 in #1


Epoch #3: 10001it [00:11, 892.39it/s, env_step=30000, gradient_step=3000, len=11, n/ep=1, n/st=10, rew=11.00]                            


...

Epoch #11: test_reward: 107.010000 ± 22.207429, best_reward: 253.650000 ± 75.944766 in #10


Epoch #12: 10001it [00:09, 1009.72it/s, env_step=120000, gradient_step=12000, len=169, n/ep=0, n/st=10, rew=169.00]                          


Epoch #12: test_reward: 182.990000 ± 24.044748, best_reward: 253.650000 ± 75.944766 in #10


Epoch #13: 10001it [00:13, 752.56it/s, env_step=130000, gradient_step=13000, len=189, n/ep=0, n/st=10, rew=189.00]                           


Epoch #13: test_reward: 496.460000 ± 18.066223, best_reward: 496.460000 ± 18.066223 in #13
New best model saved with mean reward: 496.46
Finished training in 192.78231406211853 seconds


# Mod 1: Change to cartpole-v0

In [45]:
from tianshou.env import DummyVectorEnv
# Prepare the new environment
new_env_name = "CartPole-v0"
# train_env_new = gym.make(new_env_name)
# test_env_new = gym.make(new_env_name)
train_env_new = DummyVectorEnv([lambda: gym.make(new_env_name)])
test_env_new = DummyVectorEnv([lambda: gym.make(new_env_name)])

# Update the collectors by setting their env attribute directly
# train_collector.env = train_env_new
train_collector = ts.data.Collector(
    policy, train_env_new, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True
)
test_collector.env = test_env_new

# Reset the new environments
train_collector.reset_env()
test_collector.reset_env()

# Ensure the policy is set to training mode
policy.load_state_dict(torch.load('models/cbp_dqn.pth'))
policy.train()
policy.set_eps(0.1)  # Exploration rate

# Update logger
logger = ts.utils.TensorboardLogger(SummaryWriter(f"logs/cbp_dqn_{new_env_name}"))

# Define a new train function if desired
def train_fn_new(epoch, env_step):
    eps = max(0.1, 0.5 - epoch * 0.05)
    policy.set_eps(eps)
    policy.model.update_cbp()

env_spec = gym.make(new_env_name).spec.reward_threshold

# Run the training loop
result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=10,
    step_per_epoch=5000,
    step_per_collect=10,
    episode_per_test=10,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=train_fn_new,
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= env_spec,
    logger=logger,
).run()

print(f"Finished training in {result.timing.total_time} seconds")
torch.save(policy.state_dict(), 'models/cbp_dqn_retest_v0.pth')

# --- Evaluate the trained agent ---
policy.load_state_dict(torch.load('models/cbp_dqn_retest_v0.pth'))
policy.eval()
policy.set_eps(0.05)

# Create an environment with rendering enabled.
eval_env = gym.make(env_name, render_mode="human")
eval_env.reset()
collector = ts.data.Collector(policy, eval_env, exploration_noise=True)
collector.collect(n_episode=1, render=1/35, reset_before_collect=True)
eval_env.close()

/Users/namnguyenvu/PycharmProjects/reinforcementLearning/venv/lib/python3.12/site-packages/gymnasium/envs/registration.py:523: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(
Epoch #1: 5001it [00:08, 599.72it/s, env_step=5000, gradient_step=500, len=152, n/ep=0, n/st=10, rew=152.00]                          


Epoch #1: test_reward: 102.600000 ± 39.593434, best_reward: 200.000000 ± 0.000000 in #0
Finished training in 9.72939920425415 seconds


# Mod 2: Change cartpole-v1's metrics

1. Incremental Variations of CartPole-v1 
Modify CartPole-v1's parameters gradually over time to observe how well CBP adapts:  

Gravity shift: Change gravity from 9.8 m/s² (Earth) to a lower value (e.g., Moon: 1.6 m/s²) or a higher value (e.g., Jupiter: 24.8 m/s²).  
Pole length variations: Increase or decrease length of the pole.  
Mass changes: Modify masspole or masscart dynamically every few episodes.  
Friction changes: Adjust track friction dynamically to simulate different surfaces.  
👉 Goal: See if the model continues learning and does not degrade in performance as physics conditions change.  

In [46]:
import numpy as np

class DynamicCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, gravity_range=(5.0, 25.0), length_range=(0.2, 1.0), mass_range=(0.05, 1.0), friction_range=(0.0, 0.5)):
        super().__init__(env)
        self.gravity_range = gravity_range
        self.length_range = length_range
        self.mass_range = mass_range
        self.friction_range = friction_range
        self.episode_count = 0  # Track episodes

    def reset(self, **kwargs):
        self.episode_count += 1  # Increment episode count

        # Gradually shift physics conditions over time
        new_gravity = np.interp(self.episode_count, [0, 500], self.gravity_range)  # Gradually change gravity
        new_length = np.interp(self.episode_count, [0, 500], self.length_range)  # Change pole length
        new_mass = np.interp(self.episode_count, [0, 500], self.mass_range)  # Change pole mass
        new_friction = np.interp(self.episode_count, [0, 500], self.friction_range)  # Change friction

        # Apply new parameters
        self.env.env.gravity = new_gravity
        self.env.env.length = new_length
        self.env.env.masspole = new_mass
        self.env.env.force_mag = 10.0 * (new_mass / 0.1)  # Scale force with mass to balance difficulty
        self.env.env.friction = new_friction

        print(f"Episode {self.episode_count}: Gravity={new_gravity}, Length={new_length}, Mass={new_mass}, Friction={new_friction}")

        return self.env.reset(**kwargs)

env_name = "CartPole-v1"
base_env = gym.make(env_name)
train_env = DynamicCartPoleWrapper(base_env)

test_env = DynamicCartPoleWrapper(gym.make(env_name))  # Use same variations for testing

train_collector = ts.data.Collector(
    policy, train_env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True
)
test_collector = ts.data.Collector(policy, test_env, exploration_noise=True)

# Load the pre-trained model from Code Block 1 
policy.load_state_dict(torch.load('models/cbp_dqn.pth')) 
policy.eval()

best_mean_reward = -np.inf 

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(policy.state_dict(), 'models/cbp_dqn_v1_alternate_metrics.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

# Train the model while CBP adapts to the changing environment
result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=30,
    step_per_epoch=10000,
    step_per_collect=10,
    episode_per_test=100,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=ts.utils.TensorboardLogger(SummaryWriter("logs/cbp_dynamic_cartpole")),
).run()

save_best_model(result.best_reward)

Episode 1: Gravity=5.04, Length=0.2016, Mass=0.0519, Friction=0.001
Episode 1: Gravity=5.04, Length=0.2016, Mass=0.0519, Friction=0.001
Episode 2: Gravity=5.08, Length=0.20320000000000002, Mass=0.0538, Friction=0.002
Episode 3: Gravity=5.12, Length=0.2048, Mass=0.0557, Friction=0.003
Episode 4: Gravity=5.16, Length=0.2064, Mass=0.057600000000000005, Friction=0.004
Episode 5: Gravity=5.2, Length=0.20800000000000002, Mass=0.059500000000000004, Friction=0.005
Episode 6: Gravity=5.24, Length=0.2096, Mass=0.0614, Friction=0.006
Episode 7: Gravity=5.28, Length=0.2112, Mass=0.0633, Friction=0.007
Episode 8: Gravity=5.32, Length=0.21280000000000002, Mass=0.06520000000000001, Friction=0.008
Episode 9: Gravity=5.36, Length=0.2144, Mass=0.0671, Friction=0.009000000000000001
Episode 10: Gravity=5.4, Length=0.21600000000000003, Mass=0.069, Friction=0.01
Episode 11: Gravity=5.44, Length=0.21760000000000002, Mass=0.0709, Friction=0.011
Episode 12: Gravity=5.48, Length=0.2192, Mass=0.0728, Friction=0.

Epoch #1:   1%|1         | 140/10000 [00:00<00:12, 775.21it/s, env_step=140, gradient_step=14, len=12, n/ep=0, n/st=10, rew=12.00]

Episode 3: Gravity=5.12, Length=0.2048, Mass=0.0557, Friction=0.003
Episode 4: Gravity=5.16, Length=0.2064, Mass=0.057600000000000005, Friction=0.004
Episode 5: Gravity=5.2, Length=0.20800000000000002, Mass=0.059500000000000004, Friction=0.005
Episode 6: Gravity=5.24, Length=0.2096, Mass=0.0614, Friction=0.006


Epoch #1:   2%|2         | 220/10000 [00:00<00:13, 720.37it/s, env_step=220, gradient_step=22, len=10, n/ep=1, n/st=10, rew=10.00]

Episode 7: Gravity=5.28, Length=0.2112, Mass=0.0633, Friction=0.007
Episode 8: Gravity=5.32, Length=0.21280000000000002, Mass=0.06520000000000001, Friction=0.008
Episode 9: Gravity=5.36, Length=0.2144, Mass=0.0671, Friction=0.009000000000000001
Episode 10: Gravity=5.4, Length=0.21600000000000003, Mass=0.069, Friction=0.01
Episode 11: Gravity=5.44, Length=0.21760000000000002, Mass=0.0709, Friction=0.011
Episode 12: Gravity=5.48, Length=0.2192, Mass=0.0728, Friction=0.012
Episode 13: Gravity=5.52, Length=0.22080000000000002, Mass=0.0747, Friction=0.013000000000000001


...

Episode 314: Gravity=17.560000000000002, Length=0.7024000000000001, Mass=0.6466000000000001, Friction=0.314
Episode 315: Gravity=17.6, Length=0.704, Mass=0.6485000000000001, Friction=0.315
Episode 316: Gravity=17.64, Length=0.7056, Mass=0.6504000000000001, Friction=0.316
Episode 317: Gravity=17.68, Length=0.7072, Mass=0.6523, Friction=0.317


Epoch #1:  99%|#########8| 9890/10000 [00:12<00:00, 896.76it/s, env_step=9890, gradient_step=989, len=132, n/ep=0, n/st=10, rew=132.00]

Episode 318: Gravity=17.72, Length=0.7088000000000001, Mass=0.6542, Friction=0.318


Epoch #1: 10001it [00:12, 802.29it/s, env_step=10000, gradient_step=1000, len=207, n/ep=0, n/st=10, rew=207.00]                           


Episode 319: Gravity=17.759999999999998, Length=0.7104000000000001, Mass=0.6561, Friction=0.319
Episode 207: Gravity=13.28, Length=0.5312, Mass=0.44329999999999997, Friction=0.20700000000000002
Episode 208: Gravity=13.32, Length=0.5328, Mass=0.4452, Friction=0.20800000000000002
Episode 209: Gravity=13.36, Length=0.5344, Mass=0.4471, Friction=0.209
Episode 210: Gravity=13.4, Length=0.536, Mass=0.449, Friction=0.21
Episode 211: Gravity=13.44, Length=0.5376000000000001, Mass=0.45089999999999997, Friction=0.211
Episode 212: Gravity=13.48, Length=0.5392, Mass=0.4528, Friction=0.212
Episode 213: Gravity=13.52, Length=0.5408, Mass=0.4547, Friction=0.213
Episode 214: Gravity=13.56, Length=0.5424, Mass=0.4566, Friction=0.214
Episode 215: Gravity=13.6, Length=0.544, Mass=0.45849999999999996, Friction=0.215
Episode 216: Gravity=13.64, Length=0.5456000000000001, Mass=0.4604, Friction=0.216
Episode 217: Gravity=13.68, Length=0.5472, Mass=0.4623, Friction=0.217
Episode 218: Gravity=13.72, Length=0.5

2. More Complex or Noisy CartPole Variants  
To evaluate adaptation beyond gradual parameter shifts:  

Noisy CartPole: Introduce small random perturbations to forces applied by the agent.  
Windy CartPole: Apply lateral forces that vary over time.  
Delayed-Action CartPole: Introduce a small delay between the agent’s action and its effect.  

👉 Goal: Test CBP’s ability to handle unpredictability in dynamic environments.

In [47]:
import numpy as np

class NoisyCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, noise_std=0.1):
        super().__init__(env)
        self.noise_std = noise_std  # Standard deviation of noise

    def step(self, action):
        noisy_action = action + np.random.normal(0, self.noise_std)  # Add Gaussian noise
        noisy_action = np.clip(noisy_action, 0, 1)  # Ensure action remains valid
        return self.env.step(int(noisy_action))  # Convert back to integer action


class WindyCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, wind_max=5.0, wind_change_prob=0.1):
        super().__init__(env)
        self.wind_max = wind_max  # Max lateral force
        self.wind_force = 0
        self.wind_change_prob = wind_change_prob  # Probability of wind change per step

    def step(self, action):
        # Occasionally change wind force
        if np.random.rand() < self.wind_change_prob:
            self.wind_force = np.random.uniform(-self.wind_max, self.wind_max)

        # Get the step results
        obs, reward, terminated, truncated, info = self.env.step(action)
        
        # Apply wind effect by modifying the observation directly
        # CartPole observation: [position, velocity, pole_angle, pole_angular_velocity]
        obs[1] += self.wind_force * 0.01  # Modify cart velocity in the observation
        
        # Update the environment's state array
        self.env.unwrapped.state = obs.copy()

        return obs, reward, terminated, truncated, info

from collections import deque

class DelayedActionCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, delay_steps=3):
        super().__init__(env)
        self.delay_steps = delay_steps  # How many steps before action takes effect
        self.action_queue = deque([0] * delay_steps, maxlen=delay_steps)  # Store past actions

    def step(self, action):
        self.action_queue.append(action)  # Queue the new action
        delayed_action = self.action_queue.popleft()  # Use delayed action
        return self.env.step(delayed_action)  # Will automatically pass through all 5 values

train_env = WindyCartPoleWrapper(NoisyCartPoleWrapper(DelayedActionCartPoleWrapper(gym.make("CartPole-v1"))))
test_env = WindyCartPoleWrapper(NoisyCartPoleWrapper(DelayedActionCartPoleWrapper(gym.make("CartPole-v1"))))

# policy.load_state_dict(torch.load('models/cbp_dqn_adapted_v1.pth')) # Should change name to cbp_dqn_adapted_v1.pth, but haven't
policy.load_state_dict(torch.load('models/cbp_dqn_v1_alternate_metrics.pth'))
policy.train()
policy.set_eps(0.1)  # Start with exploration

best_mean_reward = -np.inf

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(policy.state_dict(), 'models/cbp_dqn_final_adapted_v2.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=ts.data.Collector(policy, train_env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True),
    test_collector=ts.data.Collector(policy, test_env, exploration_noise=True),
    max_epoch=20,
    step_per_epoch=5000,
    step_per_collect=10,
    episode_per_test=20,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(max(0.1, 0.5 - epoch * 0.05)),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=ts.utils.TensorboardLogger(SummaryWriter("logs/cbp_windy_noisy_delayed")),
).run()

print(f"Finished training in {result.timing.total_time} seconds")
save_best_model(result.best_reward)

Epoch #1: 5001it [00:07, 644.27it/s, env_step=5000, gradient_step=500, len=14, n/ep=0, n/st=10, rew=14.00]                          


Epoch #1: test_reward: 22.300000 ± 8.933644, best_reward: 22.300000 ± 8.933644 in #1


Epoch #2: 5001it [00:06, 754.27it/s, env_step=10000, gradient_step=1000, len=28, n/ep=1, n/st=10, rew=28.00]                          


Epoch #2: test_reward: 24.600000 ± 14.416657, best_reward: 24.600000 ± 14.416657 in #2


Epoch #3: 5001it [00:07, 659.75it/s, env_step=15000, gradient_step=1500, len=21, n/ep=0, n/st=10, rew=21.00]                          


...

Epoch #18: test_reward: 25.200000 ± 11.227644, best_reward: 35.450000 ± 24.303241 in #9


Epoch #19: 5001it [00:05, 933.59it/s, env_step=95000, gradient_step=9500, len=27, n/ep=1, n/st=10, rew=27.00]                           


Epoch #19: test_reward: 27.150000 ± 14.054448, best_reward: 35.450000 ± 24.303241 in #9


Epoch #20: 5001it [00:05, 984.16it/s, env_step=100000, gradient_step=10000, len=17, n/ep=0, n/st=10, rew=17.00]                          


Epoch #20: test_reward: 30.200000 ± 13.969968, best_reward: 35.450000 ± 24.303241 in #9
Finished training in 128.2709560394287 seconds
New best model saved with mean reward: 35.45


# Changes to improve  
1️⃣ NoisyCartPoleWrapper

✅ Correct Approach: Adds Gaussian noise to the action.  
⚠ Issue:  

The action in CartPole-v1 is discrete (0 or 1), but noisy_action becomes continuous due to np.random.normal().  
Clipping (np.clip(noisy_action, 0, 1)) does not correctly map back to {0,1}.  
Fix: Instead of directly adding noise to a discrete action, use probability flipping:  

def step(self, action):  
    if np.random.rand() < self.noise_std:  # With probability `noise_std`, flip action  
        action = 1 - action  
    return self.env.step(action)  
👉 This ensures actions remain 0 or 1 while still introducing randomness.  

2️⃣ WindyCartPoleWrapper

✅ Correct Approach: Applies wind as a force that changes over time.  
⚠ Issue:  

Modifying obs[1] (cart velocity) directly does not actually apply force to the environment; it only changes the observation.  
The true physics of CartPole are managed internally, so directly changing obs[1] does not actually push the cart.  
Fix:  

Instead of modifying obs[1], add force inside the environment's physics using env.unwrapped.state.  
def step(self, action):  
    # Occasionally change wind force  
    if np.random.rand() < self.wind_change_prob:  
        self.wind_force = np.random.uniform(-self.wind_max, self.wind_max)

    # Apply action  
    obs, reward, terminated, truncated, info = self.env.step(action)

    # Apply wind effect by modifying the environment’s internal state (not just observation)  
    self.env.unwrapped.state[1] += self.wind_force  # Modify velocity directly in environment  

    return obs, reward, terminated, truncated, info  
👉 This properly simulates wind affecting the cart’s velocity.  

3️⃣ DelayedActionCartPoleWrapper

✅ Correct Approach: Introduces an action delay using a queue.  
⚠ Issue:  

The queue (deque) is initialized with 0, meaning for the first few steps the agent always takes action 0 regardless of policy.  
This biases the early learning by making the agent effectively ignore initial training steps.  
Fix:  

Instead of defaulting to 0, initialize with random past actions to avoid bias.  
self.action_queue = deque(np.random.choice([0, 1], delay_steps), maxlen=delay_steps)  
👉 This prevents early-action bias.

In [48]:
import numpy as np

class NoisyCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, noise_std=0.1):
        super().__init__(env)
        self.noise_std = noise_std  # Standard deviation of noise

    def step(self, action):
        if np.random.rand() < self.noise_std:  # With probability `noise_std`, flip action
            action = 1 - action
        return self.env.step(int(action))


class WindyCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, wind_max=5.0, wind_change_prob=0.1):
        super().__init__(env)
        self.wind_max = wind_max  # Max lateral force
        self.wind_force = 0
        self.wind_change_prob = wind_change_prob  # Probability of wind change per step

    def step(self, action):
        # Occasionally change wind force
        if np.random.rand() < self.wind_change_prob:
            self.wind_force = np.random.uniform(-self.wind_max, self.wind_max)

        # Get the step results
        obs, reward, terminated, truncated, info = self.env.step(action)
        
        # Apply wind effect by modifying the observation directly
        # CartPole observation: [position, velocity, pole_angle, pole_angular_velocity]
        obs[1] += self.wind_force * 0.01  # Modify cart velocity in the observation
        
        # Update the environment's state array
        self.env.unwrapped.state = obs.copy()

        return obs, reward, terminated, truncated, info

from collections import deque

class DelayedActionCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, delay_steps=3):
        super().__init__(env)
        self.delay_steps = delay_steps  # How many steps before action takes effect
        # self.action_queue = deque([0] * delay_steps, maxlen=delay_steps)  # Store past actions
        self.action_queue = deque(np.random.choice([0, 1], delay_steps), maxlen=delay_steps)

    def step(self, action):
        self.action_queue.append(action)  # Queue the new action
        delayed_action = self.action_queue.popleft()  # Use delayed action
        return self.env.step(delayed_action)  # Will automatically pass through all 5 values

train_env = WindyCartPoleWrapper(NoisyCartPoleWrapper(DelayedActionCartPoleWrapper(gym.make("CartPole-v1"))))
test_env = WindyCartPoleWrapper(NoisyCartPoleWrapper(DelayedActionCartPoleWrapper(gym.make("CartPole-v1"))))

# policy.load_state_dict(torch.load('models/cbp_dqn_adapted_v1.pth')) # Should change name to cbp_dqn_adapted_v1.pth, but haven't
policy.load_state_dict(torch.load('models/cbp_dqn_v1_alternate_metrics.pth'))
policy.train()
policy.set_eps(0.1)  # Start with exploration

best_mean_reward = -np.inf

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(policy.state_dict(), 'models/cbp_dqn_final_adapted_v2.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=ts.data.Collector(policy, train_env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True),
    test_collector=ts.data.Collector(policy, test_env, exploration_noise=True),
    max_epoch=30,
    step_per_epoch=5000,
    step_per_collect=10,
    episode_per_test=20,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(max(0.1, 0.5 - epoch * 0.05)),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=ts.utils.TensorboardLogger(SummaryWriter("logs/cbp_windy_noisy_delayed")),
).run()

print(f"Finished training in {result.timing.total_time} seconds")
save_best_model(result.best_reward)

Epoch #1: 5001it [00:04, 1047.77it/s, env_step=5000, gradient_step=500, len=45, n/ep=0, n/st=10, rew=45.00]                            


Epoch #1: test_reward: 41.650000 ± 9.477737, best_reward: 107.550000 ± 69.976049 in #0


Epoch #2: 5001it [00:04, 1086.11it/s, env_step=10000, gradient_step=1000, len=27, n/ep=0, n/st=10, rew=27.00]                          


Epoch #2: test_reward: 20.200000 ± 5.921149, best_reward: 107.550000 ± 69.976049 in #0


Epoch #3: 5001it [00:05, 958.49it/s, env_step=15000, gradient_step=1500, len=29, n/ep=0, n/st=10, rew=29.00]                            


...

Epoch #28: test_reward: 25.550000 ± 8.120807, best_reward: 113.200000 ± 83.819807 in #8


Epoch #29: 5001it [00:04, 1050.56it/s, env_step=145000, gradient_step=14500, len=71, n/ep=0, n/st=10, rew=71.00]                            


Epoch #29: test_reward: 34.800000 ± 9.130170, best_reward: 113.200000 ± 83.819807 in #8


Epoch #30: 5001it [00:04, 1064.66it/s, env_step=150000, gradient_step=15000, len=25, n/ep=0, n/st=10, rew=25.00]                            


Epoch #30: test_reward: 43.250000 ± 27.356672, best_reward: 113.200000 ± 83.819807 in #8
Finished training in 161.48106002807617 seconds
New best model saved with mean reward: 113.2


In [49]:
import numpy as np

class NoisyCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, noise_std=0.1):
        super().__init__(env)
        self.noise_std = noise_std  # Standard deviation of noise

    def step(self, action):
        if np.random.rand() < self.noise_std:  # With probability `noise_std`, flip action
            action = 1 - action
        return self.env.step(int(action))


class WindyCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, wind_max=5.0, wind_change_prob=0.1):
        super().__init__(env)
        self.wind_max = wind_max  # Max lateral force
        self.wind_force = 0
        self.wind_change_prob = wind_change_prob  # Probability of wind change per step

    def step(self, action):
        # Occasionally change wind force
        if np.random.rand() < self.wind_change_prob:
            self.wind_force = np.random.uniform(-self.wind_max, self.wind_max)

        # Get the step results
        obs, reward, terminated, truncated, info = self.env.step(action)
        
        # Apply wind effect by modifying the observation directly
        # CartPole observation: [position, velocity, pole_angle, pole_angular_velocity]
        obs[1] += self.wind_force * 0.01  # Modify cart velocity in the observation
        
        # Update the environment's state array
        self.env.unwrapped.state = obs.copy()

        return obs, reward, terminated, truncated, info

from collections import deque

class DelayedActionCartPoleWrapper(gym.Wrapper):
    def __init__(self, env, delay_steps=3):
        super().__init__(env)
        self.delay_steps = delay_steps  # How many steps before action takes effect
        # self.action_queue = deque([0] * delay_steps, maxlen=delay_steps)  # Store past actions
        self.action_queue = deque(np.random.choice([0, 1], delay_steps), maxlen=delay_steps)

    def step(self, action):
        self.action_queue.append(action)  # Queue the new action
        delayed_action = self.action_queue.popleft()  # Use delayed action
        return self.env.step(delayed_action)  # Will automatically pass through all 5 values

train_env = WindyCartPoleWrapper(NoisyCartPoleWrapper(DelayedActionCartPoleWrapper(gym.make("CartPole-v1"))))
test_env = WindyCartPoleWrapper(NoisyCartPoleWrapper(DelayedActionCartPoleWrapper(gym.make("CartPole-v1"))))

# policy.load_state_dict(torch.load('models/cbp_dqn_adapted_v1.pth')) # Should change name to cbp_dqn_adapted_v1.pth, but haven't
policy.load_state_dict(torch.load('models/cbp_dqn_v1_alternate_metrics.pth'))
policy.train()
policy.set_eps(0.1)  # Start with exploration

best_mean_reward = -np.inf

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(policy.state_dict(), 'models/cbp_dqn_final_adapted_v2.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

result = ts.trainer.OffpolicyTrainer(
    policy=policy,
    train_collector=ts.data.Collector(policy, train_env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True),
    test_collector=ts.data.Collector(policy, test_env, exploration_noise=True),
    max_epoch=30,
    step_per_epoch=5000,
    step_per_collect=10,
    episode_per_test=20,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: policy.set_eps(max(0.1, 0.5 - epoch * 0.05)),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=ts.utils.TensorboardLogger(SummaryWriter("logs/cbp_windy_noisy_delayed")),
).run()

print(f"Finished training in {result.timing.total_time} seconds")
save_best_model(result.best_reward)

Epoch #1: 5001it [00:04, 1014.47it/s, env_step=5000, gradient_step=500, len=27, n/ep=0, n/st=10, rew=27.00]                            


Epoch #1: test_reward: 45.850000 ± 16.429470, best_reward: 120.050000 ± 69.162472 in #0


Epoch #2: 5001it [00:05, 875.09it/s, env_step=10000, gradient_step=1000, len=26, n/ep=0, n/st=10, rew=26.00]                           


Epoch #2: test_reward: 32.250000 ± 8.269674, best_reward: 120.050000 ± 69.162472 in #0


Epoch #3: 5001it [00:07, 674.21it/s, env_step=15000, gradient_step=1500, len=119, n/ep=0, n/st=10, rew=119.00]                          


...

Epoch #28: test_reward: 96.700000 ± 57.258275, best_reward: 120.050000 ± 69.162472 in #0


Epoch #29: 5001it [00:04, 1007.36it/s, env_step=145000, gradient_step=14500, len=136, n/ep=0, n/st=10, rew=136.00]                          


Epoch #29: test_reward: 113.400000 ± 55.955697, best_reward: 120.050000 ± 69.162472 in #0


Epoch #30: 5001it [00:05, 908.73it/s, env_step=150000, gradient_step=15000, len=93, n/ep=0, n/st=10, rew=93.00]                             


Epoch #30: test_reward: 49.200000 ± 41.820569, best_reward: 120.050000 ± 69.162472 in #0
Finished training in 219.03963708877563 seconds
New best model saved with mean reward: 120.05


3. Transfer to a Different Control Problem 
For a drastic test of adaptation, switch to an environment requiring similar skills but with new challenges:  

MountainCarContinuous-v0: Requires continuous control and memory of past actions, unlike CartPole’s immediate balance task.  
Pendulum-v1: Similar physics, but requires torque control instead of discrete left/right actions.  
Acrobot-v1: A more complex version of inverted pendulum, with two linked segments.  
👉 Goal: Test if CBP allows knowledge retention while adapting to a new but related task.  

Notes:  
Question 1:
Theory: Agent w/o backprop 
And compare to Agent with backprop  
Graph of: train in first, 2nd, 3rd, etc  

Question 2: to address, which env work well and which doesn't .   
-> Maze: change end point (up mean down, down mean up)


Think about complexity: how behavior change  
Jump cartpole to other env  

Question 3: Does continual backprop work with larger inputs  

In [50]:
# First create a new policy with correct dimensions for Acrobot
env_name = "Acrobot-v1"
train_env = gym.make(env_name)
test_env = gym.make(env_name)

input_dim = train_env.observation_space.shape[0]  # Should be 6
output_dim = train_env.action_space.n  # Should be 3

# Create a new model with correct dimensions
new_model = CBPNetwork(
    input_dim=input_dim, 
    output_dim=output_dim, 
    hidden_sizes=[128, 128, 128], 
    cbp_params={'eta': 0.99, 'rho': 1e-4, 'm': 100}
)

# Create a new policy
new_policy = CBPDQNPolicy(
    model=new_model,
    optim=torch.optim.Adam(new_model.parameters(), lr=0.001),
    discount_factor=0.975,
    estimation_step=3,
    target_update_freq=1000,
    action_space=train_env.action_space
)

# Load the pretrained weights for matching layers
pretrained_dict = torch.load("models/cbp_dqn_final_adapted_v2.pth")
model_dict = new_policy.state_dict()

# Filter out incompatible layers
filtered_dict = {k: v for k, v in pretrained_dict.items() 
                if k in model_dict and 
                'hidden_layers.0' not in k and  # Skip first layer
                'out_layer' not in k and        # Skip output layer
                model_dict[k].shape == v.shape}

# Update matching layers
model_dict.update(filtered_dict)
new_policy.load_state_dict(model_dict)

# Now you can train with the new policy
new_policy.train()
new_policy.set_eps(0.1)

best_mean_reward = -np.inf

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(new_policy.state_dict(), 'models/cbp_dqn_Acrobot-v1.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

result = ts.trainer.OffpolicyTrainer(
    policy=new_policy,
    train_collector=ts.data.Collector(new_policy, train_env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True),
    test_collector=ts.data.Collector(new_policy, test_env, exploration_noise=True),
    max_epoch=20,
    step_per_epoch=5000,
    step_per_collect=10,
    episode_per_test=20,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: new_policy.set_eps(max(0.1, 0.5 - epoch * 0.05)),
    test_fn=lambda epoch, env_step: new_policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=ts.utils.TensorboardLogger(SummaryWriter("logs/cbp_Acrobot-v1")),
).run()

print(f"Finished training in {result.timing.total_time} seconds")
save_best_model(result.best_reward)

Epoch #1: 5001it [00:05, 894.52it/s, env_step=5000, gradient_step=500, len=392, n/ep=0, n/st=10, rew=-391.00]                           


Epoch #1: test_reward: -500.000000 ± 0.000000, best_reward: -500.000000 ± 0.000000 in #0


Epoch #2: 5001it [00:06, 821.09it/s, env_step=10000, gradient_step=1000, len=500, n/ep=0, n/st=10, rew=-500.00]                          


Epoch #2: test_reward: -301.000000 ± 78.367085, best_reward: -301.000000 ± 78.367085 in #2


Epoch #3: 5001it [00:04, 1007.88it/s, env_step=15000, gradient_step=1500, len=500, n/ep=0, n/st=10, rew=-500.00]                          


...

Epoch #18: test_reward: -500.000000 ± 0.000000, best_reward: -301.000000 ± 78.367085 in #2


Epoch #19: 5001it [00:04, 1072.03it/s, env_step=95000, gradient_step=9500, len=500, n/ep=0, n/st=10, rew=-500.00]                          


Epoch #19: test_reward: -500.000000 ± 0.000000, best_reward: -301.000000 ± 78.367085 in #2


Epoch #20: 5001it [00:04, 1054.89it/s, env_step=100000, gradient_step=10000, len=500, n/ep=0, n/st=10, rew=-500.00]                          


Epoch #20: test_reward: -500.000000 ± 0.000000, best_reward: -301.000000 ± 78.367085 in #2
Finished training in 166.45523834228516 seconds
New best model saved with mean reward: -301.0


In [51]:
print(new_policy)

CBPDQNPolicy(
  (model): CBPNetwork(
    (hidden_layers): ModuleList(
      (0): CBPLayer(
        (linear): Linear(in_features=6, out_features=128, bias=True)
        (activation): ReLU()
      )
      (1-2): 2 x CBPLayer(
        (linear): Linear(in_features=128, out_features=128, bias=True)
        (activation): ReLU()
      )
    )
    (out_layer): Linear(in_features=128, out_features=3, bias=True)
  )
  (model_old): CBPNetwork(
    (hidden_layers): ModuleList(
      (0): CBPLayer(
        (linear): Linear(in_features=6, out_features=128, bias=True)
        (activation): ReLU()
      )
      (1-2): 2 x CBPLayer(
        (linear): Linear(in_features=128, out_features=128, bias=True)
        (activation): ReLU()
      )
    )
    (out_layer): Linear(in_features=128, out_features=3, bias=True)
  )
)


In [53]:
# First create a new policy with correct dimensions for Acrobot
env_name = "MountainCar-v0"
train_env = gym.make(env_name)
test_env = gym.make(env_name)

input_dim = train_env.observation_space.shape[0]  # Should be 6
output_dim = train_env.action_space.n  # Should be 3

# Create a new model with correct dimensions
new_model = CBPNetwork(
    input_dim=input_dim, 
    output_dim=output_dim, 
    hidden_sizes=[128, 128, 128], 
    cbp_params={'eta': 0.99, 'rho': 1e-4, 'm': 100}
)

# Create a new policy
new_policy = CBPDQNPolicy(
    model=new_model,
    optim=torch.optim.Adam(new_model.parameters(), lr=0.001),
    discount_factor=0.975,
    estimation_step=3,
    target_update_freq=1000,
    action_space=train_env.action_space
)

# Load the pretrained weights for matching layers
pretrained_dict = torch.load("models/cbp_dqn_final_adapted_v2.pth")
model_dict = new_policy.state_dict()

# Filter out incompatible layers
filtered_dict = {k: v for k, v in pretrained_dict.items() 
                if k in model_dict and 
                'hidden_layers.0' not in k and  # Skip first layer
                'out_layer' not in k and        # Skip output layer
                model_dict[k].shape == v.shape}

# Update matching layers
model_dict.update(filtered_dict)
new_policy.load_state_dict(model_dict)

# Now you can train with the new policy
new_policy.train()
new_policy.set_eps(0.1)

best_mean_reward = -np.inf

def save_best_model(mean_rewards):
    global best_mean_reward
    if mean_rewards > best_mean_reward:
        best_mean_reward = mean_rewards
        torch.save(new_policy.state_dict(), 'models/cbp_dqn_MountainCar-v0.pth')
        print(f"New best model saved with mean reward: {best_mean_reward}")

result = ts.trainer.OffpolicyTrainer(
    policy=new_policy,
    train_collector=ts.data.Collector(new_policy, train_env, ts.data.VectorReplayBuffer(20000, 1), exploration_noise=True),
    test_collector=ts.data.Collector(new_policy, test_env, exploration_noise=True),
    max_epoch=20,
    step_per_epoch=5000,
    step_per_collect=10,
    episode_per_test=20,
    batch_size=64,
    update_per_step=1 / 10,
    train_fn=lambda epoch, env_step: new_policy.set_eps(max(0.1, 0.5 - epoch * 0.05)),
    test_fn=lambda epoch, env_step: new_policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= train_env.spec.reward_threshold,
    logger=ts.utils.TensorboardLogger(SummaryWriter("logs/cbp_MountainCar-v0")),
).run()

print(f"Finished training in {result.timing.total_time} seconds")
save_best_model(result.best_reward)

Epoch #1: 5001it [00:05, 988.06it/s, env_step=5000, gradient_step=500, len=200, n/ep=1, n/st=10, rew=-200.00]                          


Epoch #1: test_reward: -200.000000 ± 0.000000, best_reward: -200.000000 ± 0.000000 in #0


Epoch #2: 5001it [00:04, 1078.50it/s, env_step=10000, gradient_step=1000, len=200, n/ep=1, n/st=10, rew=-200.00]                          


Epoch #2: test_reward: -200.000000 ± 0.000000, best_reward: -200.000000 ± 0.000000 in #0


Epoch #3: 5001it [00:04, 1037.31it/s, env_step=15000, gradient_step=1500, len=200, n/ep=1, n/st=10, rew=-200.00]                          


...

Epoch #18: test_reward: -200.000000 ± 0.000000, best_reward: -200.000000 ± 0.000000 in #0


Epoch #19: 5001it [00:04, 1077.44it/s, env_step=95000, gradient_step=9500, len=200, n/ep=0, n/st=10, rew=-200.00]                          


Epoch #19: test_reward: -200.000000 ± 0.000000, best_reward: -200.000000 ± 0.000000 in #0


Epoch #20: 5001it [00:05, 919.22it/s, env_step=100000, gradient_step=10000, len=200, n/ep=0, n/st=10, rew=-200.00]                          


Epoch #20: test_reward: -200.000000 ± 0.000000, best_reward: -200.000000 ± 0.000000 in #0
Finished training in 122.64696025848389 seconds
New best model saved with mean reward: -200.0
